#Paquetes necesarios

In [1]:
import cv2  
import numpy as np
import matplotlib.pyplot as plt

TAREA: Captura una o varias imágenes con monedas no solapadas y algún objeto que no lo sea. Filtra los contornos que no se correpondan con monedas, y muestra el número total de monedas presentes en la imagen.

In [11]:
cap = cv2.VideoCapture(0)

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_color = (0, 0, 255)
font_thickness = 2

while True:
    ret, frame = cap.read()

    # Convierte el frame a escala de grises
    frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Aplica un filtro de suavizado para reducir el ruido
    frame_gris = cv2.GaussianBlur(frame_gris, (5, 5), 0)

    # Umbral adaptativo para mejorar la segmentación
    img_th1 = cv2.adaptiveThreshold(frame_gris, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Detectar bordes usando Canny para mejorar la precisión en los bordes
    edges = cv2.Canny(frame_gris, 100, 200)

    # Obtiene contornos de los bordes detectados
    contornos, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_cont = np.zeros(img_rgb.shape)
    filtered_contours = 0

    for c in contornos:
        # Área del contorno
        area = cv2.contourArea(c)
        
        # Filtramos por área mínima (ajustar según el tamaño de las monedas)
        if area > 100:  # Puedes ajustar este valor dependiendo del tamaño de las monedas
            perimetro = cv2.arcLength(c, True)
            
            # Filtramos por perímetro máximo
            if perimetro < 2000:  # Puedes ajustar este valor dependiendo del tamaño de las monedas
                # Añadimos al contador
                filtered_contours += 1
                
                # Dibuja los contornos en blanco (de fondo) y contornos en verde
                cv2.drawContours(img_cont, [c], -1, (255, 255, 255), -1)
                cv2.drawContours(img_rgb, [c], -1, (0, 255, 0), 4)

    total_coins = filtered_contours  # Actualiza el total de monedas

    # Dibuja el número total de monedas en la imagen
    cv2.putText(frame, 'Numero total de monedas: {}'.format(total_coins), (10, 30), font, font_scale, font_color, font_thickness)

    # Muestra las imágenes en una pantalla dividida
    cv2.imshow('Webcam', frame)
    cv2.imshow('Contornos', img_rgb)

    # Condición de salida con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera la captura de video y cierra las ventanas
cap.release()
cv2.destroyAllWindows()